In [1]:
import numpy as np
import pandas as pd

In [2]:
Geography = np.zeros(62,dtype = 'object')
income_mean = np.zeros((62,7))
income_median = np.zeros((62,7))
total_pop = np.zeros((62,7))
black_pop = np.zeros((62,7))
white_pop = np.zeros((62,7))
black_rate = np.zeros((62,7))
white_rate = np.zeros((62,7))

In [3]:
file_list = ['Income/ACS_11_5YR_S1901_with_ann.csv','Income/ACS_12_5YR_S1901_with_ann.csv','Income/ACS_13_5YR_S1901_with_ann.csv',
             'Income/ACS_14_5YR_S1901_with_ann.csv','Income/ACS_15_5YR_S1901_with_ann.csv','Income/ACS_16_5YR_S1901_with_ann.csv',
             'Income/ACS_17_5YR_S1901_with_ann.csv']
file_list2 = ['Population/ACS_11_5YR_B02001_with_ann.csv','Population/ACS_12_5YR_B02001_with_ann.csv','Population/ACS_13_5YR_B02001_with_ann.csv',
              'Population/ACS_14_5YR_B02001_with_ann.csv','Population/ACS_15_5YR_B02001_with_ann.csv','Population/ACS_16_5YR_B02001_with_ann.csv',
              'Population/ACS_17_5YR_B02001_with_ann.csv']
for i in range(6):
    data1 = pd.read_csv(file_list[i],header = None, skiprows = [0,1], names = ['zipcode','income median','income mean'], usecols = [2,91,99])
    data2 = pd.read_csv(file_list2[i],header = None, skiprows = [0,1], names = ['zipcode', 'total', 'white', 'black'], usecols = [2,3,5,7])
    for j in range(62):
        Geography[j] = data1['zipcode'][j]
        income_mean[j,i] = data1['income mean'][j]
        income_median[j,i] = data1['income median'][j]
        total_pop[j,i] = data2['total'][j]
        white_pop[j,i] = data2['white'][j]
        black_pop[j,i] = data2['black'][j]
        white_rate[j,i] = white_pop[j,i]/total_pop[j,i]
        black_rate[j,i] = black_pop[j,i]/total_pop[j,i]
data1 = pd.read_csv(file_list[6],header = None, skiprows = [0,1], names = ['zipcode','income median','income mean'], usecols = [2,25,27])
data2 = pd.read_csv(file_list2[6],header = None, skiprows = [0,1], names = ['zipcode', 'total', 'white', 'black'], usecols = [2,3,5,7])
for j in range(62):
    Geography[j] = data1['zipcode'][j]
    income_mean[j,6] = data1['income mean'][j]
    income_median[j,6] = data1['income median'][j]
    total_pop[j,6] = data2['total'][j]
    white_pop[j,6] = data2['white'][j]
    black_pop[j,6] = data2['black'][j]
    white_rate[j,6] = white_pop[j,6]/total_pop[j,6]
    black_rate[j,6] = black_pop[j,6]/total_pop[j,6]


In [5]:
def loss(b1, b2, r):
    Loss = []
    def pr(x1, x2, x3, x4):
        u1 = b1*x1 + b2*x2
        u2 = b1*x1 + b2*x2
        diff = u2-u1
        return 1/(1+np.exp(-r*diff))
    for i in range(6):
        probability = np.zeros((62,62))
        black_pop_end = np.zero(62)
        black_pop_end_true = black_pop[:,i+1]
        income_start = income_mean[:,i]
        black_rate_start = black_rate[:,i]
        black_pop_start = black_pop[:,i]
        for j in range(62):
            for k in range(62):
                x1, x2, x3, x4 = income_start[j], black_rate_start[j], income_start[k], black_rate_start[k]
                probability[j,k] = pr(x1, x2, x3, x4)
        for q in range(62):
            total = probability[q,:].sum()
            probability[q,:] = probability[q,:]/total
        for p in range(62):
            List = list(range(62)).pop(p)
            black_pop_end[p] = (probability[:,p]*black_pop_start).sum()
        loss = ((black_pop_end - black_pop_end_true)**2).sum()
        Loss.append(loss)
    return np.array(Loss).sum()